In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [16]:
## List of pandas dataframes
all_results = []
num_threads = [1, 2, 4, 8, 16, 32]

In [17]:
final_folder = 'final_results/'
# get all folders in the final_results directory
all_result_folders = os.listdir(final_folder)


In [28]:
def is_concurrent_perf(file_path:str)-> bool:
    """
    Check if the file is a concurrent performance result file.
    :param file: file name
    :return: True if the file is a concurrent performance result file, False otherwise
    """
    return file_path.startswith('concurrent') and file_path.endswith('_perf.txt')

def is_independent_perf(file_path:str)-> bool:
    """
    Check if the file is a concurrent performance result file.
    :param file: file name
    :return: True if the file is a concurrent performance result file, False otherwise
    """
    return file_path.startswith('independent') and file_path.endswith('_perf.txt')

def is_hyper(file_path: str)-> bool:
    """
    Check if the file is a hyper performance result file.
    :param file: file name
    :return: True if the file is a hyper performance result file, False otherwise
    """
    return 'hyper' in file_path


In [ ]:
current_folder = all_perf_result_folders[0]

# Go into the folder and get all txt files
folder_path = os.path.join(final_folder, current_folder)
all_txt_files = os.listdir(folder_path)        
# Only keep the txt files
concurrent_txt_files = [file for file in all_txt_files if is_concurrent_perf(file)]
indpendent_txt_files = [file for file in all_txt_files if is_independent_perf(file)]
    
    

In [ ]:
def read_value_key(line: str)-> tuple[str, int, float]:
    """
    Read the value and key from the file name.
    :param read_file: file name
    :return: tuple of value and key
    """
    # Split the file name by '_'
    words = line.split()
    # if 'instructions' in words:

    
    count = words[0]
    key = words[1] 
    # Get the last word (the percentage) and remove the parentheses to parse it to a float
    percentage = words[-1].strip("()%")
    return key, int(count), float(percentage)


In [93]:
def read_perf_file(current_folder: str, perf_txt_file: str)-> pd.DataFrame:
    """
    """
    columns = ['key', 'count', 'percentage']
    df = pd.DataFrame(columns=columns)
    current_file_path = final_folder + '/' + current_folder + '/' + perf_txt_file
    print(current_file_path, end=' ')
    with open(current_file_path, 'r') as read_file:
        # all_lines = read_file.readlines()
        # for i, line in enumerate(all_lines):
        #     print(f"Line {i}: {line}")
        # get rid of the first 5 lines
        _ = read_file.readline()
        _ = read_file.readline()
        _ = read_file.readline()
        _ = read_file.readline()
        _ = read_file.readline()

        for i in range(10):
            line = read_file.readline()
            try:
                key, count, percentage = read_value_key(line)
                df.loc[len(df)] = [key, count, percentage]
            except ValueError as e:
                print(f"Error reading line {i}: {e}")
        _ = read_file.readline()
        line12 = read_file.readline()
        print(line12)
        return df

In [ ]:
# Only keep the folders that start with 'perf_' 
all_perf_result_folders = [folder for folder in all_result_folders if folder.startswith('perf_') and not folder.endswith('.txt')]
for perf_result_folder in all_perf_result_folders:

    # Go into the folder and get all txt files
    folder_path = os.path.join(final_folder, perf_result_folder)
    all_txt_files = os.listdir(folder_path)        
    # Only keep the txt files
    concurrent_txt_files = [file for file in all_txt_files if is_concurrent_perf(file)]
    indpendent_txt_files = [file for file in all_txt_files if is_independent_perf(file)]
    
    for concurrent_txt_file in concurrent_txt_files:
        # Read the file and store it in a dataframe
        df = read_perf_file(perf_result_folder, concurrent_txt_file)
        current_file = os.path.join(perf_result_folder, concurrent_txt_file)
        df.to_csv()

final_results//perf_2025-03-27_09-54-24_numa_0/concurrent_numa_0_1_perf.txt 

final_results//perf_2025-03-27_09-54-24_numa_0/concurrent_numa_0_4_perf.txt 

final_results//perf_2025-03-27_09-54-24_numa_0/concurrent_numa_0_32_perf.txt 

final_results//perf_2025-03-27_09-54-24_numa_0/concurrent_numa_0_16_perf.txt 

final_results//perf_2025-03-27_09-54-24_numa_0/concurrent_numa_0_8_perf.txt 

final_results//perf_2025-03-27_09-54-24_numa_0/concurrent_numa_0_2_perf.txt 

final_results//perf_2025-03-26_10-03-57_hyper/concurrent_hyper_threading_16_perf.txt 

final_results//perf_2025-03-26_10-03-57_hyper/concurrent_hyper_threading_1_perf.txt 

final_results//perf_2025-03-26_10-03-57_hyper/concurrent_hyper_threading_2_perf.txt 

final_results//perf_2025-03-26_10-03-57_hyper/concurrent_hyper_threading_4_perf.txt 

final_results//perf_2025-03-26_10-03-57_hyper/concurrent_hyper_threading_32_perf.txt 

final_results//perf_2025-03-26_10-03-57_hyper/concurrent_hyper_threading_8_perf.txt 

final_result

In [87]:
df

,key,count,percentage
0,cycles,308497836792,25.01
1,instructions,286372457339,30.01
2,L1-icache-load-misses,740725975,30.01
3,L1-dcache-load-misses,1640611579,30.02
4,LLC-load-misses,71526064,30.02
5,cache-misses,416908007,30.02
6,uops_retired.stall_cycles,188670203930,20.01
7,branch-misses,1723468290,20.02
8,iTLB-load-misses,4946671,20.02
9,dTLB-load-misses,81765450,20.01
